# Cours TAL
## Master Humanités Numériques – PSL

## [Spacy](https://spacy.io)

- Bibliothèque logicielle de TAL écrite en Python (et Cython)
- Étiquetage POS, lemmatisation, analyse syntaxique, entités nommées, word embedding
- Usage de modèles neuronaux
- Intégration aisée de bibliothèques de deep learning
- v2.2.3 ([github](https://github.com/explosion/spaCy))
- Licence MIT (Open Source) pour le code
    - Licences ouvertes diverses pour les modèles
- Produit de la société [explosion.ai](https://explosion.ai/). Fondé par : Matthew Honnibal ([@honnibal](https://twitter.com/honnibal)) et Ines Montani ([@_inesmontani](https://twitter.com/_inesmontani))

## Pourquoi Spacy ?

- C'est du Python 🙌 🎉
- Plutôt simple à prendre en main
- Très bien documenté, à notre avis. D'ailleurs plutôt que ce notebook, suivez l'excellent tutorial d'Ines Montani : [https://course.spacy.io/](https://course.spacy.io/)
- Couvre les traitements d'une chaîne de TAL typique
- Pas mal utilisé dans l'industrie
- MAIS ce n'est pas forcément l'outil qui donne les meilleurs résultats pour le français


## Spacy et les autres

Spacy est *un* des frameworks de TAL disponibles

- [NLTK](http://www.nltk.org/) : python, orienté pédagogie, pas de modèles neuronaux inclus mais se combine bien avec TensorFlow, PyTorch ou AlleNLP
- [Stanford Core NLP](https://stanfordnlp.github.io/stanfordnlp/) : java, modèles pour 53 langues (UD), résolution de la coréference.
    <small>[https://github.com/explosion/spacy-stanfordnlp](https://github.com/explosion/spacy-stanfordnlp) permet d'utiliser les modèles de CoreNLP avec Spacy</small>
- [TextBlob](https://textblob.readthedocs.io/en/dev/)
- [DKPro](https://dkpro.github.io/)
- [flair](https://github.com/zalandoresearch/flair)

## installation

dans un terminal
```bash
python3 -m pip install --user spacy 
#ou pip install --user spacy
```
- installation du modèle français
```bash
python3 -m spacy download fr_core_news_sm
#ou python3 -m spacy download fr_core_news_md
```
- vérification
```bash
python3 -m spacy validate
```


## modèles

- 10 langues : anglais, allemand, français, espagnol, portugais, italien, néerlandais, grec, norvégien, lituanien + modèle multi langues
- 2 modèles pour le français
    - fr_core_news_sm (tagger, parser, ner) 14 Mo
    - fr_core_news_md (tagger, parser, ner, vectors) 84 Mo
- modèles `fr` appris sur les corpus [Sequoia](https://deep-sequoia.inria.fr/fr/) et [WikiNer](https://figshare.com/articles/Learning_multilingual_named_entity_recognition_from_Wikipedia/5462500)


## usage

- *si vous voulez utiliser Spacy prenez le temps de lire la [documentation](https://spacy.io/usage), ici ce ne sera qu'un coup d'œil incomplet*
- un modèle est une instance de la classe `Language`, il est adapté à une langue en particulier
- un modèle incorpore un vocabulaire, des poids, des vecteurs de mots, une configuration

In [ ]:
import spacy
nlp = spacy.load('fr_core_news_sm')

In [ ]:
nlp

## usage

- le traitement fonctionne avec un [*pipeline*](https://spacy.io/usage/spacy-101#pipelines) pour convertir un texte en objet `Doc` (texte annoté)
- par défaut `tokenizer` > `tagger` > `parser` > `ner` > `…`
- l'utilisateur peut ajouter des étapes ou en retrancher

In [ ]:
nlp = spacy.load('fr_core_news_sm', disable=["parser", "ner"])

## usage

 - Un objet `Doc` est une séquence d'objets `Token` (voir l'[API](https://spacy.io/api/token))
 - Le texte d'origine est découpé en phrases, tokenizé, annoté en POS, lemme, syntaxe (dépendance) et en entités nommées (NER)

## usage – tokenization

La tokenization de Spacy est non-destructive. Vous pouvez découper un texte en tokens et le restituer dans sa forme originale.

In [ ]:
doc = nlp("L’Organisation des Nations unies (ONU) a lancé mardi un appel d’urgence pour lever des dizaines de millions de dollars afin de protéger les réfugiés vulnérables face à la propagation du nouveau coronavirus.")
for tok in doc:
    print(tok)

In [ ]:
for tok in doc:
    print(tok.text_with_ws, end="")

## usage – étiquetage

In [ ]:
doc = nlp("Dans les derniers jours de mai 1793, un des bataillons parisiens amenés en Bretagne par Santerre fouillait le redoutable bois de la Saudraie en Astillé. On n'était pas plus de trois cents, car le bataillon était décimé par cette rude guerre.")
for token in doc:
    print(token.text, token.tag_, token.lemma_, token.ent_type_)

Pour traiter plusieurs textes, vous pouvez utiliser [nlp.pipe](https://spacy.io/api/language#pipe)


```python
docs = list(nlp.pipe(texts))
```

## usage – NER

Accès direct aux entités de l'objet `Doc`

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
doc = nlp('Le président Xi Jinping a affirmé que la propagation du coronavirus était « pratiquement jugulée ». Il s’est d’ailleurs rendu pour la première fois à Wuhan, la capitale de la province du Hubei, le berceau du Covid-19.')
displacy.render(doc, style="ent", jupyter=True)

## usage – analyse syntaxique

In [ ]:
displacy.render(doc, style="dep", jupyter=True, options={'distance':70})

`noun_chunks` permet de récupérer les syntagmes nominaux d'un document

In [ ]:
for chunk in doc.noun_chunks:
    print(f"{chunk.text} ({chunk.root.text})")

On peut aussi récupérer la tête syntaxique et ses dépendants

In [ ]:
root = [token for token in doc if token.head == token][0]
subjects = [tok for tok in root.lefts if tok.dep_ == "nsubj"]
subject = subjects[0]

In [ ]:
for descendant in subject.subtree:
    print(descendant.text)

## À vous

1. Trouver et afficher l'objet de la phrase : « Depuis que Google a annoncé son intention de stopper d'ici deux ans les cookies tiers sur Chrome , son moteur de recherche qui est utilisé par plus de 60 % de la population mondiale connectée, les Criteo, LiveRamp et autres Index Exchange se préparent à ce qui peut être considéré comme un séisme, à leur échelle.»

2. Afficher les entités nommées et leur catégorie de cette même phrase. Y-a-t'il des erreurs selon vous ?

## Adapter les traitements de Spacy

## 1. re-tokenisation

- voir [https://spacy.io/usage/linguistic-features#retokenization](https://spacy.io/usage/linguistic-features#retokenization)

In [ ]:
doc = nlp("Pour les bons bails ça va grave quer-cra")
print([tok.text for tok in doc])

In [ ]:
with doc.retokenize() as retokenizer:
    retokenizer.merge(doc[7:], attrs={"LEMMA": "quer-cra"})
print([tok.text for tok in doc])

## 2. Entités nommées : traitement par règles
 - Voir [https://spacy.io/usage/rule-based-matching#entityruler](https://spacy.io/usage/rule-based-matching#entityruler)

In [ ]:
from spacy.pipeline import EntityRuler
nlp = spacy.load('fr_core_news_sm')
doc = nlp("Depuis que Google a annoncé son intention de stopper d'ici deux ans les cookies tiers sur Chrome , son moteur de recherche qui est utilisé par plus de 60 % de la population mondiale connectée, les Criteo, LiveRamp et autres Index Exchange se préparent à ce qui peut être considéré comme un séisme, à leur échelle.")
print("Avant : ", [(ent.text, ent.label_) for ent in doc.ents])
# ('Moutain View', MISC), ('Stadia', LOC)
ruler = EntityRuler(nlp, overwrite_ents=True)
patterns = [{"label": "ORG", "pattern": "Chrome"},
    {"label":"ORG", "pattern":"Criteo"},
    {"label":"ORG","pattern":"LiveRamp"}]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)
doc = nlp("Depuis que Google a annoncé son intention de stopper d'ici deux ans les cookies tiers sur Chrome , son moteur de recherche qui est utilisé par plus de 60 % de la population mondiale connectée, les Criteo, LiveRamp et autres Index Exchange se préparent à ce qui peut être considéré comme un séisme, à leur échelle.")
print("Après : ", [(ent.text, ent.label_) for ent in doc.ents])

## 3. Entités nommées  : adaptation du modèle

In [ ]:
from spacy.util import minibatch, compounding
from pathlib import Path
import random
# training data
TRAIN_DATA = [
    ("Criteo fonctionne grâce à des data centers", {"entities": [(0, 6, "ORG")]}),
    ("LiveRamp a racheté Criteo", {"entities": [(0, 8, "ORG"), (18, 25, "ORG")]}),
]

nlp = spacy.load('fr_core_news_sm')
print("Original model :")
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])


ner = nlp.get_pipe("ner")
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(100):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print("Losses", losses)

# test the trained model
print("Testing model : ")
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

# save model to output directory
output_dir = Path("./")
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
